# Exercise - Multi step Workflow - SOLUTION

In this exercise, you’ll build a multi-step workflow using LCEL to solve a more complex task than simply generating a joke. 

**Challenge**

Create an AI Business Advisor that:

1. Accepts an industry as input.
2. Generates a business idea.
3. Analyzes the strengths and weaknesses.
4. Formats the results as a final report.


## 0. Import the necessary libs

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")
base_url = os.getenv("OPENAI_ENDPOINT")
model_name = "gpt-4o-mini"
temp=0.0

llm = ChatOpenAI(
    base_url=base_url,
    api_key=api_key,
    model=model_name,
    temperature=temp
)

## 1. Instantiate Chat Model with your API Key

## 2. Your first Chain

In the end of each chain, you should parse the output and save the logs

In [ ]:
logs = []

In [ ]:
parser = StrOutputParser()

In [ ]:
parse_and_log_output_chain = RunnableParallel(
    output=parser, 
    log=RunnableLambda(lambda x: logs.append(x))
)

## 3. Idea Generation

Craft a prompt to generate a business idea for the given industry. 

Make sure {industry} placeholder is inside your template, so it can be filled when the chain is invoked.

In [ ]:
idea_prompt = PromptTemplate(
    template=(
        "You are a creative business advisor. "
        "Generate one innovative business idea in the industry: "
        "{industry}. "
        "Provide a brief description of the idea."
    )
)

In [ ]:
idea_chain = (
    idea_prompt 
    | llm 
    | parse_and_log_output_chain
)

In [ ]:
idea_result = idea_chain.invoke("agro")

In [ ]:
idea_result["output"]

In [ ]:
logs

## 4. Idea Analysis

Craft a prompt to analyze the generated idea's strengths and weaknesses

In [ ]:
analysis_prompt = PromptTemplate(
    template=(
        "Analyze the following business idea: "
        "Idea: {idea} "
        "Identify 3 key strengths and 3 potential weaknesses of the idea."
    )
)

In [ ]:
analysis_chain = (
    analysis_prompt 
    | llm 
    | parse_and_log_output_chain
)

In [ ]:
analysis_result = analysis_chain.invoke(idea_result["output"])

In [ ]:
analysis_result["output"]

In [ ]:
logs

## 5. Report Generation

Craft a prompt to structure the information into a business report.

In [ ]:
report_prompt = PromptTemplate(
    template=(
        "Here is a business analysis: "
        "Strengths & Weaknesses: {output} "
        "Generate a structured business report." 
    )
)

In [ ]:
class AnalysisReport(BaseModel):
    """Strengths and Weaknesses about a business idea"""
    strengths: list = Field(default=[], description="Idea's strength list")
    weaknesses: list = Field(default=[], description="Idea's weaknesse list")

In [ ]:
report_chain = (
    report_prompt | llm.with_structured_output(schema=AnalysisReport, method="function_calling")
)

In [ ]:
report_result = report_chain.invoke(analysis_result["output"])

In [ ]:
report_result

## 6. End to End Chain

In [ ]:
e2e_chain = ( 
    RunnablePassthrough() 
    | idea_chain
    | RunnableParallel(idea=RunnablePassthrough())
    | analysis_chain
    | report_chain
)

In [ ]:
e2e_chain.get_graph().print_ascii()

In [ ]:
# Change the industry if you want
e2e_result = e2e_chain.invoke("agro")

In [ ]:
e2e_result

In [ ]:
e2e_result.strengths

In [ ]:
e2e_result.weaknesses

In [ ]:
logs

## 7. Experiment

Now that you understood how it works, experiment with new things.
- Improve memory
- Explore the Runnables
- Add More Complexity